# Imports

In [1]:
from datasets import load_dataset
import cv2
import numpy as np
import math
import json
import random
import os
from PIL import Image

# Shuffle Dataset

In [2]:
data = load_dataset("benjamin-paine/imagenet-1k-256x256")

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/36 [00:00<?, ?it/s]

In [3]:
shuffled_data = data['train'].shuffle()

In [ ]:
# nb_image = 100000
# for i, single_data in enumerate(shuffled_data):
#     if i == nb_image:
#         break
#     image = single_data["image"].convert('RGB')
#     image = np.array(image)[:, :, ::-1].copy()
#
#     image = cv2.resize(image, (640, 640), interpolation=cv2.INTER_LINEAR)
#
#     cv2.imshow("", image)
#     cv2.waitKey(0)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is not the object's thread (0x316cf560).
Cannot move to target thread (0x31272610)

QObject::moveToThread: Current thread (0x31272610) is n

# Choose random cards

In [4]:
with open("card_sets.json", "r") as f:
    card_sets = json.load(f)

In [46]:
random_card = random.choice(list(card_sets.keys()))
random_card

'Hiita-the-Fire-Charmer-0-759393'

In [47]:
path = os.path.join("datasets/Zouloux", random_card, random_card.split('-')[-1] + ".jpg")
path

'datasets/Zouloux/Hiita-the-Fire-Charmer-0-759393/759393.jpg'

In [94]:
image = Image.open(path).convert('RGB')
image = np.array(image)[:, :, ::-1].copy()

# Change image perspective

In [95]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

rotx = 20 * math.pi / 180
roty = 45 * math.pi / 180
rotz = 180 * math.pi / 180

cx, cy, cz = math.cos(rotx), math.cos(roty), math.cos(rotz)
sx, sy, sz = math.sin(rotx), math.sin(roty), math.sin(rotz)

rotation = [[cz * cy, cz * sy * sx - sz * cx],
            [sz * cy, sz * sy * sx + cz * cx],
            [-sy, cy * sx]]

pt = [[-image.shape[1] / 2, -image.shape[0] / 2],
        [image.shape[1] / 2, -image.shape[0] / 2],
        [image.shape[1] / 2, image.shape[0] / 2],
        [-image.shape[1] / 2, image.shape[0] / 2]]

ptt = []
for i in range(4):
    pz = pt[i][0] * rotation[2][0] + pt[i][1] * rotation[2][1]
    ptt.append([image.shape[1] / 2 + (pt[i][0] * rotation[0][0] + pt[i][1] * rotation[0][1]) * 50 * image.shape[0] / (50 * image.shape[0] + pz),
                image.shape[0] / 2 + (pt[i][0] * rotation[1][0] + pt[i][1] * rotation[1][1]) * 50 * image.shape[0] / (50 * image.shape[0] + pz)])

in_pt = np.array([
    [0, 0],
    [image.shape[1], 0],
    [image.shape[1], image.shape[0]],
    [0, image.shape[0]]
],np.float32)
out_pt = np.array(ptt,np.float32)

transform = cv2.getPerspectiveTransform(in_pt, out_pt)

In [96]:
image = cv2.warpPerspective(image, transform, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue = [255, 255, 255, 0])

In [97]:
cv2.imshow("", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is n

# Blend image

In [53]:
background = next(iter(shuffled_data))["image"].convert('RGB')
background = np.array(background)[:, :, ::-1].copy()
background = cv2.resize(background, (640, 640), interpolation=cv2.INTER_LINEAR)

In [99]:
image = cv2.resize(image, (74, 102), interpolation=cv2.INTER_LINEAR)

In [56]:
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [64]:
ret, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

In [101]:
def alphaMerge(small_foreground, background, top, left):

    result = background.copy()

    fg_b, fg_g, fg_r, fg_a = cv2.split(small_foreground)

    fg_a = fg_a / 255.0

    label_rgb = cv2.merge([fg_b * fg_a, fg_g * fg_a, fg_r * fg_a])

    height, width = small_foreground.shape[0], small_foreground.shape[1]
    part_of_bg = result[top:top + height, left:left + width, :]

    bg_b, bg_g, bg_r = cv2.split(part_of_bg)

    part_of_bg = cv2.merge([bg_b * (1 - fg_a), bg_g * (1 - fg_a), bg_r * (1 - fg_a)])

    cv2.add(label_rgb, part_of_bg, part_of_bg)

    result[top:top + height, left:left + width, :] = part_of_bg
    return result

In [102]:
result = alphaMerge(image, background, 100, 200)

In [103]:
cv2.imshow("", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is not the object's thread (0x2fd60a50).
Cannot move to target thread (0x306d2760)

QObject::moveToThread: Current thread (0x306d2760) is n